In [1]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

Using TensorFlow backend.


In [6]:
# we create two instances with the same arguments
data_gen_args = dict(rotation_range=90.,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

image_generator = image_datagen.flow_from_directory(
    'data/train/image',
    class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'data/train/label',
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

Found 5072 images belonging to 1 classes.
Found 5072 images belonging to 1 classes.


In [2]:
import os
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical  
labelencoder = LabelEncoder()
labelencoder.fit([0, 1])


LabelEncoder()

In [3]:
img_scale = (512, 512, 3)
batch_size = 8
n_label = 2

def _generateData(image_dir, label_dir=None): 
    imgnames = sorted(os.listdir(image_dir))
    labelnames = [] if label_dir is None else sorted(os.listdir(label_dir))
    
    while True:
        random_indices = np.random.choice(np.arange(len(imgnames)), batch_size)
        data = []  
        label = []
        for i in random_indices:
            image_path = os.path.join(image_dir, imgnames[i])
            image = load_img(image_path, target_size=img_scale)
            image_arr = img_to_array(image)/255 
            data.append(image_arr)

            if label_dir is not None:
                label_path = os.path.join(label_dir, labelnames[i])
                mask = load_img(label_path, target_size=img_scale, grayscale=True)
                mask_arr = img_to_array(mask).reshape((img_scale[0] * img_scale[1],))/255
                
                if np.any(mask_arr[np.where(np.logical_and(mask_arr!=0, mask_arr!=1))]):
                    print(mask_arr[np.where(np.logical_and(mask_arr!=0, mask_arr!=1))])
                    print(labelnames[i])
                if np.any(mask_arr[np.where(mask_arr!=0)] != 1):
                    print(labelnames[i])
                label.append(mask_arr)

        data = np.array(data)
        if len(label) == 0:
            yield data
        else:
            label = np.array(label).flatten()
            label = labelencoder.transform(label)
            label = to_categorical(label, num_classes=n_label)
            label = label.reshape((batch_size, img_scale[0] * img_scale[1], n_label)) 
            yield data, label


train_gen = _generateData('data/train/image/car', 'data/train/label/car')

In [4]:
imgs, masks = next(train_gen)

img, mask = imgs[0], masks[0]
img.shape, mask.shape

5056


((512, 512, 3), (262144, 2))

In [ ]:
model = load_model('_segnet.hdf5', custom_objects=dict(dice_coef=self.dice_coef))

In [64]:
img = load_img('data/train/label/car/0cdf5b5d0ce1_02_mask.gif', target_size=img_scale, grayscale=True)
arr = img_to_array(img)
arr_2d = arr.reshape(img_scale[:2])
arr_3d = np.stack((arr_2d,)*3, -1)
print(arr_3d.shape)
#new_img = array_to_img(arr_3d)
#new_img.save('aaa.gif')

(512, 512, 3)


In [52]:
train_imgnames = os.listdir('data/train/image/car')
train_labelnames = os.listdir('data/train/label/car')

labelpath = os.path.join('data/train/label/car', train_labelnames[0])
mask = load_img(labelpath, target_size=img_scale[:2], grayscale=True)
mask_arr = img_to_array(mask)
mask_arr/=255
mask_arr[np.where(mask_arr[:, :, 0]==1)]

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [43]:
for i in range(1, 17):
    binary_img = load_img('data/train/label/car/00087a6bd4dc_04_mask.jpg{num:02d}_mask.gif'.format(num=i), target_size=img_scale[:2], grayscale=True)
    binary_arr = img_to_array(binary_img)/255
    if np.all(binary_arr[np.where(binary_arr!=0)] == 1):
        print('good')
    if np.any(binary_arr[np.where(np.logical_and(binary_arr!=0, binary_arr!=1))]):
        print(binary_arr[np.where(np.logical_and(binary_arr!=0, binray_arr!=1))])
        print(labelnames[i])

good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good


In [16]:
## a1 = np.array([[e1*e2 for e1 in range(5)] for e2 in range(3)])
a2 = a1.reshape(5, -5)
print(a1)
print(a2)

[[0 0 0 0 0]
 [0 1 2 3 4]
 [0 2 4 6 8]]
[[0 0 0]
 [0 0 0]
 [1 2 3]
 [4 0 2]
 [4 6 8]]


In [35]:
print(np.random.choice(np.arange(8), 8))
print(np.random.choice(np.arange(8), 8))

[3 5 2 6 1 0 6 1]
[2 4 6 4 7 4 6 7]


In [66]:
sorted(os.listdir(None))

['.ipynb_checkpoints',
 '__pycache__',
 'data',
 'data.py',
 'datagen.ipynb',
 'gif_mask',
 'segnet.py',
 'test_predict.py',
 'unet.hdf5',
 'unet.py',
 'unknow_img.png']

In [10]:
(1, 2, 3) == tuple([e for e in range(1, 4)])

True